The data manipulation of the code, after running the converter from CEP to latitude and logitude, was done by me. For the converter from CEP to latitude and longitude, I made some adaptations from the code on this [github page]( https://github.com/mstuttgart/pycep-correios/blob/develop/pycep_correios/client.py) CEP stands for Código de Endereçamento Postal, it's the Brazilian postal code. I needed to make this conversion as part of the doctorate thesis project. This converter found approximately 80% of the latitude and longitude, for every 5 thousand CEPs, it took approximately 3 to 4 hours of processing time

# CEP to latitude and longitude

In [226]:
#base

In [227]:
import re
import string
from abc import ABC, abstractmethod

class CEPConverter(ABC):
    @abstractmethod
    def __call__(self, cep):
        pass
class Coordinates(ABC):
    def __init__(self, cep):
        self.cep = cep
        self.clean_cep = self._clean_CEP()
    def _clean_CEP(self):
        # Regex to avoid CEPs with dash ('-')
        regex = re.compile("[%s]" % re.escape(string.punctuation))
        return regex.sub("", self.cep)
    @abstractmethod
    def __call__(self):
        pass

In [228]:
#viacep

In [229]:
import json
import requests


URL_GET_ADDRESS_FROM_CEP = "http://www.viacep.com.br/ws/{}/json"


class BaseException(Exception):
    """ base exception"""

    def __init__(self, message=""):
        super(BaseException, self).__init__(message)
        self.message = message

    def __repr__(self):
        return "%s(%r)" % (self.__class__.__name__, self.message)


def get_address_from_cep(cep):
    """ Source: https://github.com/mstuttgart/pycep-correios/blob/develop/pycep_correios/client.py """
    try:
        response = requests.get(URL_GET_ADDRESS_FROM_CEP.format(cep))

        if response.status_code == 200:
            address = json.loads(response.text)

            if address.get("erro"):
                raise BaseException(message="Other error")

            return {
                "bairro": address.get("bairro", ""),
                "cep": address.get("cep", ""),
                "cidade": address.get("localidade", ""),
                "logradouro": address.get("logradouro", ""),
                "uf": address.get("uf", ""),
                "complemento": address.get("complemento", ""),
            }

        elif response.status_code == 400:
            raise BaseException(message="Invalid CEP: %s" % cep)  # noqa
        else:
            raise BaseException(message="Other error")

    except requests.exceptions.RequestException as e:
        raise BaseException(message=e)

In [230]:
#strategies

In [231]:
import os
import requests

#from .viacep import get_address_from_cep
#from .base import CEPConverter, Coordinates


class CorreiosPhotonConverter(CEPConverter):
    def __call__(self, cep):
        return CorreiosPhotonCoordinates(cep)()


class CEPAbertoConverter(CEPConverter):
    def __call__(self, cep):
        return CEPAbertoCoordinates(cep)()


class CorreiosPhotonCoordinates(Coordinates):
    def fetch_address(self):
        try:
            search_result = get_address_from_cep(self.clean_cep)
            address = " ".join(
                [
                    search_result["logradouro"],
                    search_result["bairro"],
                    search_result["cidade"],
                    search_result["uf"],
                    "Brasil",
                ]
            )
            # Treating the case when Correios API return an empty json
            if address == " Brasil":
                address = "-"
        except:
            address = "-"

        return address

    def fetch_coordinates(self, address):
        try:
            if address == "-":
                print("NaN")
                return {"latitude": float("nan"), "longitude": float("nan")}

            r = requests.get(
                "".join(["http://photon.komoot.de/api?q=", address, "&limit=1"])
            )

            result = r.json()
            # list with lon, lat
            coordinates = result["features"][0]["geometry"]["coordinates"]
            return {"latitude": coordinates[1], "longitude": coordinates[0]}
        except IndexError:
            return {"latitude": float("nan"), "longitude": float("nan")}
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)

    def __call__(self):
        address = self.fetch_address()
        coordinates = self.fetch_coordinates(address)
        return coordinates


class CEPAbertoCoordinates(Coordinates):
    def fetch_coordinates(self):
        try:
            url = f"https://www.cepaberto.com/api/v3/cep?cep={self.clean_cep}"
            # Sign up for your free token on: https://cepaberto.com/
            # export CEP_ABERTO_TOKEN='your-token'
            headers = {"Authorization": f'Token token={os.getenv("CEP_ABERTO_TOKEN")}'}
            response = requests.get(url, headers=headers)
            json_response = response.json()
            return {
                "latitude": float(json_response["latitude"]),
                "longitude": float(json_response["longitude"]),
            }
        
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        

    def __call__(self):
        coordinates = self.fetch_coordinates()
        return coordinates


In [232]:
#convert

In [233]:
#from .base import CEPConverter
#from .strategies import CorreiosPhotonConverter


def cep_to_coords(cep: str, factory: CEPConverter = CorreiosPhotonConverter) -> dict:
    coordinates = factory()(cep)
    return coordinates

In [234]:
#testing 

In [235]:
import pandas as pd

In [236]:
id1ate1000 = pd.read_excel("C:/Users/andre/Documents/Thesis/Thesis Project/Novas planilhas até 22-12-2020/cepnupitsus/id1ate1000.xlsx")

In [237]:
id1ate1000[0:100]

,id,cep
0,1,56900000
1,2,55000000
2,3,56903140
3,4,56912285
4,5,56900000
5,6,56400000
6,7,56900000
7,8,56903000
8,9,56900000
9,10,56912440


In [238]:
id1ate100 =id1ate1000.loc[0:100]

In [239]:
id1ate100

,id,cep
0,1,56900000
1,2,55000000
2,3,56903140
3,4,56912285
4,5,56900000
...,...,...
96,97,50860260
97,98,56280000
98,99,56395000
99,100,55028400


In [240]:
id1ate100.update("cep_to_coords('" + id1ate100[['cep']].astype(str) + "'),")
print(id1ate100)

      id                         cep
0      1  cep_to_coords('56900000'),
1      2  cep_to_coords('55000000'),
2      3  cep_to_coords('56903140'),
3      4  cep_to_coords('56912285'),
4      5  cep_to_coords('56900000'),
..   ...                         ...
96    97  cep_to_coords('50860260'),
97    98  cep_to_coords('56280000'),
98    99  cep_to_coords('56395000'),
99   100  cep_to_coords('55028400'),
100  101  cep_to_coords('55178000'),

[101 rows x 2 columns]


C:\Users\andre\Anaconda3\lib\site-packages\pandas\core\frame.py:6596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [241]:
id1ate100

,id,cep
0,1,"cep_to_coords('56900000'),"
1,2,"cep_to_coords('55000000'),"
2,3,"cep_to_coords('56903140'),"
3,4,"cep_to_coords('56912285'),"
4,5,"cep_to_coords('56900000'),"
...,...,...
96,97,"cep_to_coords('50860260'),"
97,98,"cep_to_coords('56280000'),"
98,99,"cep_to_coords('56395000'),"
99,100,"cep_to_coords('55028400'),"


In [242]:
id1ate100=id1ate100.loc[:,'cep']

In [243]:
id1ate100

0      cep_to_coords('56900000'),
1      cep_to_coords('55000000'),
2      cep_to_coords('56903140'),
3      cep_to_coords('56912285'),
4      cep_to_coords('56900000'),
                  ...            
96     cep_to_coords('50860260'),
97     cep_to_coords('56280000'),
98     cep_to_coords('56395000'),
99     cep_to_coords('55028400'),
100    cep_to_coords('55178000'),
Name: cep, Length: 101, dtype: object

In [244]:
print(" ".join(id1ate100))#.tolist()

cep_to_coords('56900000'), cep_to_coords('55000000'), cep_to_coords('56903140'), cep_to_coords('56912285'), cep_to_coords('56900000'), cep_to_coords('56400000'), cep_to_coords('56900000'), cep_to_coords('56903000'), cep_to_coords('56900000'), cep_to_coords('56912440'), cep_to_coords('56912190'), cep_to_coords('56840000'), cep_to_coords('56220000'), cep_to_coords('22010201'), cep_to_coords('55008020'), cep_to_coords('55660000'), cep_to_coords('55120000'), cep_to_coords('55120000'), cep_to_coords('56304920'), cep_to_coords('55160000'), cep_to_coords('53670000'), cep_to_coords('55002970'), cep_to_coords('55101000'), cep_to_coords('55000000'), cep_to_coords('55490000'), cep_to_coords('55002970'), cep_to_coords('55000000'), cep_to_coords('55490000'), cep_to_coords('55695000'), cep_to_coords('56700000'), cep_to_coords('56280000'), cep_to_coords('55030070'), cep_to_coords('55000000'), cep_to_coords('55000000'), cep_to_coords('55030450'), cep_to_coords('55435000'), cep_to_coords('56520000'), c

In [245]:
#pd.set_option('display.max_rows', None)    #to display max number of rows 
#pd.set_option('display.max_columns', None) #to display max number of columns 
#pd.set_option('display.max_rows', 100) #returns number of rows to default
#pd.set_option('display.max_columns', 100) #returns number of columns to default

In [246]:
cordenadasid1ate100 = cep_to_coords('56900000'), cep_to_coords('55000000'), cep_to_coords('56903140'), cep_to_coords('56912285'), cep_to_coords('56900000'), cep_to_coords('56400000'), cep_to_coords('56900000'), cep_to_coords('56903000'), cep_to_coords('56900000'), cep_to_coords('56912440'), cep_to_coords('56912190'), cep_to_coords('56840000'), cep_to_coords('56220000'), cep_to_coords('22010201'), cep_to_coords('55008020'), cep_to_coords('55660000'), cep_to_coords('55120000'), cep_to_coords('55120000'), cep_to_coords('56304920'), cep_to_coords('55160000'), cep_to_coords('53670000'), cep_to_coords('55002970'), cep_to_coords('55101000'), cep_to_coords('55000000'), cep_to_coords('55490000'), cep_to_coords('55002970'), cep_to_coords('55000000'), cep_to_coords('55490000'), cep_to_coords('55695000'), cep_to_coords('56700000'), cep_to_coords('56280000'), cep_to_coords('55030070'), cep_to_coords('55000000'), cep_to_coords('55000000'), cep_to_coords('55030450'), cep_to_coords('55435000'), cep_to_coords('56520000'), cep_to_coords('55020450'), cep_to_coords('55014580'), cep_to_coords('55028070'), cep_to_coords('55018640'), cep_to_coords('55018560'), cep_to_coords('57000000'), cep_to_coords('79822340'), cep_to_coords('55024590'), cep_to_coords('56300000'), cep_to_coords('55200000'), cep_to_coords('55008000'), cep_to_coords('55014635'), cep_to_coords('56180000'), cep_to_coords('50740050'), cep_to_coords('55190000'), cep_to_coords('63155000'), cep_to_coords('55670000'), cep_to_coords('55670000'), cep_to_coords('56280000'), cep_to_coords('56280000'), cep_to_coords('55819901'), cep_to_coords('52131180'), cep_to_coords('52061050'), cep_to_coords('55865000'), cep_to_coords('50620300'), cep_to_coords('55940000'), cep_to_coords('53620280'), cep_to_coords('52280050'), cep_to_coords('54300022'), cep_to_coords('55730000'), cep_to_coords('56332720'), cep_to_coords('51240200'), cep_to_coords('56250000'), cep_to_coords('56512380'), cep_to_coords('55745000'), cep_to_coords('56850000'), cep_to_coords('55720000'), cep_to_coords('55578000'), cep_to_coords('54340080'), cep_to_coords('55014265'), cep_to_coords('55860000'), cep_to_coords('63180000'), cep_to_coords('56512050'), cep_to_coords('56280000'), cep_to_coords('55270000'), cep_to_coords('53600000'), cep_to_coords('55665000'), cep_to_coords('55495000'), cep_to_coords('56163000'), cep_to_coords('50731450'), cep_to_coords('55578000'), cep_to_coords('56304620'), cep_to_coords('55720000'), cep_to_coords('55630000'), cep_to_coords('48905361'), cep_to_coords('54410280'), cep_to_coords('8062600'), cep_to_coords('55345000'), cep_to_coords('56400000'), cep_to_coords('50860260'), cep_to_coords('56280000'), cep_to_coords('56395000'), cep_to_coords('55028400'), cep_to_coords('55178000')

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


In [254]:
print(cordenadasid1ate100)

({'latitude': nan, 'longitude': nan}, {'latitude': nan, 'longitude': nan}, {'latitude': -7.981733, 'longitude': -38.2935601}, {'latitude': -7.9941181, 'longitude': -38.2850475}, {'latitude': nan, 'longitude': nan}, {'latitude': -22.3086683, 'longitude': -42.6201671}, {'latitude': nan, 'longitude': nan}, {'latitude': -7.981733, 'longitude': -38.2935601}, {'latitude': nan, 'longitude': nan}, {'latitude': nan, 'longitude': nan}, {'latitude': -7.9941321, 'longitude': -38.2876137}, {'latitude': -8.0586442, 'longitude': -34.934496}, {'latitude': -7.777592, 'longitude': -39.933797}, {'latitude': nan, 'longitude': nan}, {'latitude': -8.2872591, 'longitude': -35.9765889}, {'latitude': -8.2372132, 'longitude': -35.7558262}, {'latitude': -8.1356327, 'longitude': -35.8566421}, {'latitude': -8.1356327, 'longitude': -35.8566421}, {'latitude': -9.3938235, 'longitude': -40.4928845}, {'latitude': -8.22489615, 'longitude': -36.286024852340965}, {'latitude': -7.75353355, 'longitude': -34.9931035125}, {'l

In [255]:
id1ate100

0      cep_to_coords('56900000'),
1      cep_to_coords('55000000'),
2      cep_to_coords('56903140'),
3      cep_to_coords('56912285'),
4      cep_to_coords('56900000'),
                  ...            
96     cep_to_coords('50860260'),
97     cep_to_coords('56280000'),
98     cep_to_coords('56395000'),
99     cep_to_coords('55028400'),
100    cep_to_coords('55178000'),
Name: cep, Length: 101, dtype: object

In [271]:
#id1ate100
#id1ate100.tolist()
#print((id1ate100).tolist())

In [250]:
import re
pattern = re.compile(r"(\d+)")
result = []
for item in id1ate100.tolist():
    result.append(''.join(pattern.findall(item)))

In [256]:
print(result)

['56900000', '55000000', '56903140', '56912285', '56900000', '56400000', '56900000', '56903000', '56900000', '56912440', '56912190', '56840000', '56220000', '22010201', '55008020', '55660000', '55120000', '55120000', '56304920', '55160000', '53670000', '55002970', '55101000', '55000000', '55490000', '55002970', '55000000', '55490000', '55695000', '56700000', '56280000', '55030070', '55000000', '55000000', '55030450', '55435000', '56520000', '55020450', '55014580', '55028070', '55018640', '55018560', '57000000', '79822340', '55024590', '56300000', '55200000', '55008000', '55014635', '56180000', '50740050', '55190000', '63155000', '55670000', '55670000', '56280000', '56280000', '55819901', '52131180', '52061050', '55865000', '50620300', '55940000', '53620280', '52280050', '54300022', '55730000', '56332720', '51240200', '56250000', '56512380', '55745000', '56850000', '55720000', '55578000', '54340080', '55014265', '55860000', '63180000', '56512050', '56280000', '55270000', '53600000', '55

In [259]:
dfid1ate100 = pd.DataFrame(cordenadasid1ate100, result)

In [261]:
dfid1ate100

,latitude,longitude
56900000,NaN,NaN
55000000,NaN,NaN
56903140,-7.981733,-38.293560
56912285,-7.994118,-38.285047
56900000,NaN,NaN
...,...,...
50860260,-8.095819,-34.925165
56280000,-7.576858,-40.503851
56395000,-8.873430,-40.191597
55028400,-8.303582,-35.966518


In [262]:
dfid1ate100.reset_index(level=0, inplace=True) #make the index a column

In [263]:
dfid1ate100

,index,latitude,longitude
0,56900000,NaN,NaN
1,55000000,NaN,NaN
2,56903140,-7.981733,-38.293560
3,56912285,-7.994118,-38.285047
4,56900000,NaN,NaN
...,...,...,...
96,50860260,-8.095819,-34.925165
97,56280000,-7.576858,-40.503851
98,56395000,-8.873430,-40.191597
99,55028400,-8.303582,-35.966518


In [267]:
dfid1ate100 = dfid1ate100.rename(columns={'index':'cep'}) #change a column's name

In [268]:
dfid1ate100

,cep,latitude,longitude
0,56900000,NaN,NaN
1,55000000,NaN,NaN
2,56903140,-7.981733,-38.293560
3,56912285,-7.994118,-38.285047
4,56900000,NaN,NaN
...,...,...,...
96,50860260,-8.095819,-34.925165
97,56280000,-7.576858,-40.503851
98,56395000,-8.873430,-40.191597
99,55028400,-8.303582,-35.966518


In [272]:
dfid1ate100.to_excel('id1ate100latlong.xlsx')